In [1]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def check_url(url):
    if pd.isna(url) or str(url).strip() == "":
        return "EMPTY"

    headers = {
        "User-Agent": "VLC/3.0.18 LibVLC/3.0.18"
    }

    try:
        with requests.get(url, headers=headers, timeout=(5, 20), stream=True, allow_redirects=True, verify=False
        ) as response:
            if response.status_code in [200, 206]:
                chunk = next(response.iter_content(1024), None)
                if chunk:
                    return "WORKING"
                else:
                    return "NO_DATA"
            return response.status_code

    except Exception as e:
        return f"ERROR: {type(e).__name__}"

In [3]:
def channel_to_logo(channel: str) -> str:
    channel_slug = channel.lower().replace(" ", "-")
    return f"https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/logos/{channel_slug}.png"

In [4]:
OUTPUT_FILE = "tung_iptv.m3u"

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write("#EXTM3U\n")
    for GROUP_TITLE in ["Tennis", "Football", "Other"]:
        df = pd.read_csv(f"{GROUP_TITLE.lower()}.csv")
        for _, row in df.iterrows():
            channel = row["channel"]
            for col in [c for c in df.columns if c != 'channel']:
                url = row[col]
                if pd.isna(url) or str(url).strip() == "":
                    continue
                if check_url(url) != "WORKING":
                    continue
                f.write(
                    f'#EXTINF:-1 group-title="{GROUP_TITLE}" tvg-logo="{channel_to_logo(channel)}", '
                    f'{channel} ({col.upper()})\n'
                )
                f.write(f"{url}\n\n")